In [1]:
import pathlib

import jax
import jax.numpy as jnp
import numpy as np
from flax import nnx
from jax.sharding import PartitionSpec as P
from rte_dataset.builders import pipeline

from deeprte.configs import default
from deeprte.input_pipeline import input_pipeline_interface
from deeprte.model import features
from deeprte.model.mapping import inference_subbatch
from deeprte.model.tf import rte_features
from deeprte.train_lib import utils


In [2]:
path = "/workspaces/deeprte/ckpts/g0.1-q0.003-boundary0105"
config = default.get_config(path + "/config.yaml")

In [6]:
config.data_dir

'/workspaces/deeprte/data/tfds'

In [7]:
config.dataset_name

'rte/g0.1-gaussian'

In [12]:
config.name

'boundary'

In [3]:
devices_array = utils.create_device_mesh(config, devices=jax.local_devices())
mesh = jax.sharding.Mesh(devices_array, config.mesh_axes)

replicated_sharding = jax.sharding.NamedSharding(mesh, P(None))
data_sharding = jax.sharding.NamedSharding(mesh, P(None, *config.data_sharding))

feature_sharding = {
    k: data_sharding if k in rte_features.PHASE_COORDS_FEATURES else replicated_sharding
    for k in rte_features.FEATURES
}

rngs = jax.random.key(0)


In [4]:
(train_iter, eval_iter), data_sharding = input_pipeline_interface.create_data_iterator(
    config, mesh
)


In [10]:
batch = next(train_iter)

In [11]:
jax.tree.map(lambda x: x.shape, batch)

{'phase_coords': (8, 512, 4),
 'source': (8, 38400),
 'source_coords': (8, 38400, 4),
 'source_label': (8, 512),
 'source_weights': (8, 38400)}